In [1]:
from constants import data_path, entities_path, relations_path, Path
import pandas as pd
import seaborn as sns
import numpy as np

In [3]:
courses_df = pd.read_json(entities_path / "course.json", lines=True)

In [5]:
courses_ext_df = pd.read_parquet("mooc_cube_x-100k/course_ext.parquet.gzip")

In [15]:
course_resources = courses_ext_df[courses_ext_df["rsc_count"] > 0].explode("resource").drop(columns="about")

In [17]:
course_resources.head()

,id,name,field,prerequisites,resource,rsc_count,num_fields,field_en,Number of users
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",None,"{'chapter': '1.1.1', 'resource_id': 'V_849', '...",91,2,"[History, Chinese Language and Literature]",NaN
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",None,"{'chapter': '1.2.1', 'resource_id': 'V_850', '...",91,2,"[History, Chinese Language and Literature]",NaN
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",None,"{'chapter': '1.3.1', 'resource_id': 'V_851', '...",91,2,"[History, Chinese Language and Literature]",NaN
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",None,"{'chapter': '1.4', 'resource_id': 'Ex_856', 't...",91,2,"[History, Chinese Language and Literature]",NaN
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",None,"{'chapter': '2.1.1', 'resource_id': 'V_857', '...",91,2,"[History, Chinese Language and Literature]",NaN


In [28]:
def check_alternate_video_exc(x):
    prev_is_v = None
    for idx, row in x.iterrows():
        if row["resource"]["resource_id"].startswith("V_"):
            if prev_is_v:
                return False
            prev_is_v = True
    return prev_is_v

In [31]:
alternating_courses = course_resources.groupby("id").apply(check_alternate_video_exc)

In [36]:
np.where(alternating_courses)[0].shape

(57,)

In [46]:
alternating_course_info = courses_ext_df[courses_ext_df["id"].isin(alternating_courses.iloc[np.where(alternating_courses)[0]].reset_index()["id"])].drop(columns="about")

In [53]:
alternating_course_info[alternating_course_info["Number of users"].notna()].shape#&alternating_course_info["field"].str.len()==0].shape

(14, 9)

#### None of the alternating courses have a field, but they do have users
Could be difficult to exclusively narrow down watching behaviours related to only those fields. Need to verify if video ids are unique within a course. They probably are.

14 Courses with enrolled students and alternating video-exercise.